Dataset Summary - Regression:

This dataset is a set of test scores from students, along with various facts about the students that were collected. The goal of this set is to predict the scores of the students (In this limiting case, the Math Scores) with a regression-based algorithm from scikit. 

Data Explanation:<br>
Gender: 1 for male, 0 for female <br>
EthnicGroup: A through D encoded to 0 - 3<br>
ParentEduc: High school through Masters degree, 0 - 4 rankings<br>
LunchType: 1 for standard, 0 for free/reduced lunch<br>
TestPrep: 1 for none, 0 for completed<br>
ParentMaritalStatus: 0 for divorced, 1 for single, 2 for married, 3 for widowed<br>
PracticeSport: 0 for never, 1 for regularly, 2 for sometimes<br>
IsFirstChild: 1 for yes, 0 for no<br>
NrSiblings: Value<br>
TransportMeans: 1 for school bus, 0 for private<br>
WklyStudyHours: 0 for 5 to 10, 1 for less than 5, 2 for greater than 10<br>
MathScore, ReadingScore, WritingScore: Value (range of 0 - 100)<br>

In [5]:
# Import for model dumping later
import pickle

In [6]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor


In [7]:
scores_te = pd.read_csv("""datasets_training/regressiondataset_final_toml.csv""", delimiter=',')
scores_te = scores_te.drop("Unnamed: 0", axis=1) #Data needed to be dropped, dropped from original file anyway
scores_te.head(10)

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0,1,4.0,1,1,2,2,1,4.0,1,1,87,93,91
1,1,2,2.0,1,1,1,2,1,0.0,1,0,76,78,75
2,0,1,1.0,1,1,1,1,1,1.0,1,0,73,84,79
3,0,1,2.0,1,0,3,0,0,1.0,0,0,85,93,89
4,1,1,2.0,0,1,1,2,1,1.0,0,2,41,43,39
5,1,3,0.0,0,0,2,2,0,3.0,0,2,65,64,68
6,1,3,1.0,1,1,0,2,1,1.0,1,0,40,52,43
7,0,1,0.0,1,1,1,1,0,1.0,0,0,66,82,74
8,1,0,2.0,1,0,2,2,1,1.0,0,2,80,73,71
9,0,0,4.0,1,1,0,2,1,2.0,0,1,48,53,58


Splitting the dataset: <br>
The dataset was split 80/20, which is a standard splitting, as you can see in test_size below. I followed the part 3 example as well as several online explanations.

In [8]:
import json

y = scores_te["MathScore"]



#drop1 = scores_te.drop("MathScore", axis=1)
#drop2 = drop1.drop("WritingScore", axis=1)
#drop3 = drop2.drop("MathScore", axis=1)

x = scores_te.drop("MathScore", axis=1)

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42) #as per part 3 example

model = RandomForestRegressor(random_state=42)  

model.fit(x_train, y_train)

y_pred_inital = (model.predict(x_test))

# Hmmmm

y_pred = json.dumps((y_pred_inital.tolist()))
#y_pred = np.array(y_pred_inital)
#print(y_pred[1])
print(x_test[1]) #These are scaled, I guess. 13 of them ~ note.

#These lines may have been through a few iterations without dumping the pkl file


[-0.9806177  -1.04269426 -1.28644445  0.73755442  0.72690488  1.29371956
  0.88760283 -1.34202922  0.58871739  0.83739685  0.49353854 -0.78506797
 -0.37815958]


Algorithm Choices/Choice Made: <br>
The algorithms provided by scikit are plenty, so I decided to sort through them, running this code through each to see which algorithm delivered the best eval model as seen below. I eventually settled on a random forest regressor model - while it preformed somewhat similarly to a few competitive models, it has much less overfitting as per the random forest models in general.

Ways to evalulate the model:

In [9]:
#Eval

mse = mean_squared_error(y_test, y_pred_inital)
print("mean squared error for the Math Score Algorithm:", mse)

rsquared = r2_score(y_test, y_pred_inital)
print("R^2 for the Math Score Algorithm(accuracy metric):", rsquared)


mean squared error for the Math Score Algorithm: 32.834743042348414
R^2 for the Math Score Algorithm(accuracy metric): 0.8621014325281838


Summary:
In summary, for this part, a machine learning model was trained and exported based on the data collected and refined from the preprocessing segment. The algoritm was chosen (RFR) and evalulated using mean squared error and R squared. It preformed well, with an R squared score measuring close to .9 depending on the iteration. Finally, the model will be exported using a pkl file for Azure.

Note:
The model right now is trained for scores in the Math segment, although by modifying the lines in the training section, any of the segments could be predicted.

In [10]:
# Model dumping
pickle.dump(model, open("reg_model.pkl", "wb")) #Only run once per model iteration (if there are multiple)